In [61]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import re

In [2]:
class car_thief:
    '''car_thief() will output a pandas dataframe containing data on used cars, from https://www.cazoo.co.uk/.
            - use method fetch_links() for getting links to all car adverts on the website.
    '''
    def __init__(self):
        
        self.headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
        self.results_price = []
        self.all_links = []
        self.soup_menu = []
        
        self.price = []
        self.name = []
        self.year = []
        self.transmission = []
        self.mileage = []
        self.seats = []
        self.engine = []
        self.body = []
        self.colour = []
        self.drive = []
        self.registration = []
        self.previous_owners = []
        self.keys = []
        self.emissions = []
        self.ulez = []
        self.top_speed= []
        self.accelaration_0_62 = []
        self.power = []
        self.tax = []
        self.insurance = []
        self.mpg = []
        self.service_record = []
        
    def fetch_links(self, pages):
        ''' fetch_links will creat list containing links to all car adverts and store in object,
            - function argument is number of pages on website.
            - use method soup_chef() next, to scrape data from each link .'''
        
        hrefs = []
        prices = []
        for i in range(1,pages):
            url = 'https://www.cazoo.co.uk/cars/?page='+ str(i)

            response = requests.get(url, headers = self.headers)

            soup = BeautifulSoup(response.content, 'html.parser')
            
            link = soup.find_all('a', {'class' : "vehicle-cardstyles__LinkWrap-sc-1bxv5iu-4 dLzowz"})
            price = soup.find_all('div', {'class':"pricingstyles__Wrap-rs9839-0 ckMUIu"})
            
            links.append(link)
            results_price.append(price)
        
        clean_hrefs = []
        for i in hrefs:
            for j in i:
                car = j.get("href")
                clean_hrefs.append(car)
        
        for i in range(len(clean_hrefs)):
            clean_hrefs[i] = "https://www.cazoo.co.uk" + clean_hrefs[i]
        
        self.all_links.append(clean_hrefs)
        self.results_price.append(prices)
        
    def soup_chef(self):
        ''' 
                        ~~~~~~~~~~~~~ THIS WILL TAKE A WHILE!!~~~~~~~~~~~~~~~~
            soup_chef() will parse all the car advert htmls, and make soup objects to be parsed later for all data, 
            the output will stored in object.
            - use soup_critic() to parse soup objects and extract all data which will be stored in object'''
        
        car_soups = []
        for i in all_links:
            response = requests.get(i, headers = self.headers)
            soup = BeautifulSoup(response.content, 'html.parser')
            
            car_soups.append(soup)
        
        
        self.soup_menu.append(car_soups)
        
    def soup_critic(self):
        '''soup_critic() will parse all soup objects and extract data on used cars.All data will be stored in object.
           -  use panda_tamer() next to make a pandas dataframe containing all data'''
            
        for l,page in enumerate(self.results_price):
            for k,i in enumerate(page):
                p = i.find('p', {'class': 'full-pricestyles__Price-sc-12l0fhp-0 kZsXtM','data-test-id':"card-pricing-full-price-gb"})
                
                if p != None:
                    self.price.append(p.get_text())
        
                elif p == None:
                    clean = results_price[l][k].get_text() #get all text from each div section
                    value = re.findall('(£\d+(\,)?\d+\d){2}',clean)[0][0]# will return the second pattern match, the current price
                    self.price.append(value)
                
        
        for soup in car_soups:
            car = soup.find("h1",{"class":"sc-yrk414-0 Abyeg"}).get_text()
            name.append(car)
            specs =["registrationDate","gearbox","odometerReading","numSeats","engineSize","bodyType","exteriorColour","driveType","vrm","numberOfPreviousOwners","numberOfFunctioningKeys","emissions","ulezChargeExempt","topSpeedMph","from0to62milesAccelerationInSec","enginePowerBhp","vehicleTaxPerYear","insurance","fuelConsumption"]
            
            details = []
            
            for spec in specs:
                indv = soup.find("div",{"class": "flex flex-wrap gap-x-xs py-s","data-test-id": spec})
                
                if indv != None:
                    details.append(indv.find("dd",{"class": "font-semibold"}).get_text())
                else:
                    details.append(np.NaN)
    
            self.year.append(details[0])
            self.transmission.append(details[1])
            self.mileage.append(details[2])
            self.seats.append(details[3])
            self.engine.append(details[4])
            self.body.append(details[5])
            self.colour.append(details[6])
            self.drive.append(details[7])
            self.registration.append(details[8])
            self.previous_owners.append(details[9])
            self.keys.append(details[10])
            self.emissions.append(details[11])
            self.ulez.append(details[12])
            self.top_speed.append(details[13])
            self.accelaration_0_62.append(details[14])
            self.power.append(details[15])
            self.tax.append(details[16])
            self.insurance.append(details[17])
            self.mpg.append(details[18])
    
            service = soup.find("section",{"class":"px-s md:px-l self-center w-full max-w-base","id": "service-history"})
            if service != None:
                number = service.find_all("article",{"class":"AccordionItemstyles__Wrap-sc-1i7ks8d-0 erdWGR"})
                self.service_record.append(len(number))
            else:
                self.service_record.append(0)
                
    def panda_tamer(self):
        ''' panda tamer will output all car data in pandas DataFrame form. save output onto a variable'''
        
        df = pd.DataFrame({'Name': name,'Year': year, 'Mileage': mileage, 'Engine_Size': engine, 
                           'Transmission': transmission, 'Seats': seats , 'Body_Type': body, "Colour": colour, 'Drive_Type': drive,
                           'Registration': registration, 'Previous_Owners': previous_owners, 'Keys': keys, 'Emissions':emissions, 
                           'ULEZ_Exempt':ulez,'Top_speed':top_speed, 'Accelaration_0_62': accelaration_0_62, 'Power':power, 'Tax': tax,
                           'Insurance': insurance, 'Mpg': mpg, 'Service_record' : service_record,'Price': cost})
        
        '''There will be cars that were sold while the scraper was running'''
        missing_col = df.drop(columns = ['Service_record','Price','Name']) # dataframe with columns containing no Nan
        all_NaN = missing_col.isnull().all(1) #row indexes where all columns in missing_col contain NaN
        name_Nans = cazoo_cars.loc[all_NaN, "Name"] # name values for rows with all NaN
        missing_col[all_NaN].join(name_Nans).head()
        
        df.drop(cazoo_cars[all_NaN].index, inplace = True)
        df.reset_index(drop = True, inplace = True)
        
        return df
    
        

The pandas dataframe obtained from car_thief is not entirely clean. panda_cleaner can do an initial clean, which will improve the dataset.

In [ ]:
class panda_cleaner:
    ''' Initial data from the scraper will have some issues. 
         - cars being sold while scraper is running causing data to be wiped out.
         - formatting issues
         - presence of problematic symbols
         
         This class will clean the data, input dataframe as argument and use methods when neccessary.
         
         - Regex commands are used to extract string patterns, change as required.'''
    def __init__(self,df):
        self.df = df
    
    def drop_sold_cars(self):
        '
        missing_col = self.df.drop(columns = ['Service_record','Price','Name']) # dataframe with columns containing no Nan
        all_NaN = missing_col.isnull().all(1) #row indexes where all columns in missing_col contain NaN
        name_Nans = cazoo_cars.loc[all_NaN, "Name"] # name values for rows with all NaN
        missing_col[all_NaN].join(name_Nans).head()
        
        self.df.drop(cazoo_cars[all_NaN].index, inplace = True)
        self.df.reset_index(drop = True, inplace = True)
    

Car details are stored in a webpage  that is accesed via a href on the 'vehicle card' the following code will extract the hyperlinks along with the car's price. While the price can be included within the hyperlink, in most cases there is no price displayed if the car is being sold. Due to this fact, the price shall be axtracted from the main results page. The full price is stored in different classes for each results card, therefore this scraper will extract a general `div` section and use a different for loop to extract price data.

In [33]:
links = []
results_price = []
for i in range(1,161):
    
    url = 'https://www.cazoo.co.uk/cars/?page='+ str(i)
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

    response = requests.get(url, headers = headers)

    soup = BeautifulSoup(response.content, 'html.parser')
            
    link = soup.find_all('a', {'class' : "vehicle-cardstyles__LinkWrap-sc-1bxv5iu-4 dLzowz"})
    price = soup.find_all('div', {'class':"pricingstyles__Wrap-rs9839-0 ckMUIu"})
            
    links.append(link)
    results_price.append(price)
        
        

In [43]:
all_links = []
for i in links:
    for link in i:
        car = link.get("href")
        all_links.append(car)
        

In [44]:
for i in range(len(all_links)):
    all_links[i] = "https://www.cazoo.co.uk" + all_links[i]

In [47]:
car_soups = []
for i in all_links:
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

    response = requests.get(i, headers = headers)
    
    soup = BeautifulSoup(response.content, 'html.parser')
    
    car_soups.append(soup)
    

Using `i.find('p', {'class': 'full-pricestyles__Price-sc-12l0fhp-0 kZsXtM'})` to extract price data threw out some None values, this class usualy contains the full price data. Investigating these cases revealed that some cards did not store price data in the this class,  it was stored in a new variable, to show that the price has been reduced , it will be necessary to extract all the text from these cases and and use regex to extract the last price mentioned, which corrosponds to the cars current value.

In [75]:
cost = []

for l,page in enumerate(results_price):
    for k,i in enumerate(page):
        
        p = i.find('p', {'class': 'full-pricestyles__Price-sc-12l0fhp-0 kZsXtM','data-test-id':"card-pricing-full-price-gb"})
 
        if p != None:
            cost.append(p.get_text())
        
        elif p == None:
            clean = results_price[l][k].get_text() #get all text from each div section
            value = re.findall('(£\d+(\,)?\d+\d){2}',clean)[0][0]# will return the second pattern match, the current price
            cost.append(value)
        

In [127]:

name = []
year = []
transmission = []
mileage = []
seats = []
engine = []
body = []
colour = []
drive = []
registration = []
previous_owners = []
keys = []
emissions = []
ulez = []
top_speed= []
accelaration_0_62 = []
power = []
tax = []
insurance = []
mpg = []
service_record = []

for soup in car_soups:
    car = soup.find("h1",{"class":"sc-yrk414-0 Abyeg"}).get_text()
    name.append(car)
    specs =["registrationDate","gearbox","odometerReading","numSeats","engineSize","bodyType","exteriorColour","driveType","vrm","numberOfPreviousOwners","numberOfFunctioningKeys","emissions","ulezChargeExempt","topSpeedMph","from0to62milesAccelerationInSec","enginePowerBhp","vehicleTaxPerYear","insurance","fuelConsumption"
       ]
    details = []
    for spec in specs:
        indv = soup.find("div",{"class": "flex flex-wrap gap-x-xs py-s","data-test-id": spec})
        if indv != None:
            details.append(indv.find("dd",{"class": "font-semibold"}).get_text())
        else:
            details.append(np.NaN)
    year.append(details[0])
    transmission.append(details[1])
    mileage.append(details[2])
    seats.append(details[3])
    engine.append(details[4])
    body.append(details[5])
    colour.append(details[6])
    drive.append(details[7])
    registration.append(details[8])
    previous_owners.append(details[9])
    keys.append(details[10])
    emissions.append(details[11])
    ulez.append(details[12])
    top_speed.append(details[13])
    accelaration_0_62.append(details[14])
    power.append(details[15])
    tax.append(details[16])
    insurance.append(details[17])
    mpg.append(details[18])
    
    service = soup.find("section",{"class":"px-s md:px-l self-center w-full max-w-base","id": "service-history"})
    if service != None:
        number = service.find_all("article",{"class":"AccordionItemstyles__Wrap-sc-1i7ks8d-0 erdWGR"})
        service_record.append(len(number))
    else:
        service_record.append(0)

range(0, 18)

In [544]:
cazoo_cars = pd.DataFrame({'Name': name,'Year': year, 'Mileage': mileage, 'Engine_Size': engine, 
                           'Transmission': transmission, 'Seats': seats , 'Body_Type': body, "Colour": colour, 'Drive_Type': drive,
                           'Registration': registration, 'Previous_Owners': previous_owners, 'Keys': keys, 'Emissions':emissions, 
                           'ULEZ_Exempt':ulez,'Top_speed':top_speed, 'Accelaration_0_62': accelaration_0_62, 'Power':power, 'Tax': tax,
                           'Insurance': insurance, 'Mpg': mpg, 'Service_record' : service_record,'Price': cost})

In [545]:
cazoo_cars.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7520 entries, 0 to 7519
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Name               7520 non-null   object
 1   Year               7509 non-null   object
 2   Mileage            7509 non-null   object
 3   Engine_Size        7381 non-null   object
 4   Transmission       7509 non-null   object
 5   Seats              7509 non-null   object
 6   Body_Type          7509 non-null   object
 7   Colour             7509 non-null   object
 8   Drive_Type         7508 non-null   object
 9   Registration       7509 non-null   object
 10  Previous_Owners    7509 non-null   object
 11  Keys               7509 non-null   object
 12  Emissions          7509 non-null   object
 13  ULEZ_Exempt        7505 non-null   object
 14  Top_speed          7509 non-null   object
 15  Accelaration_0_62  7509 non-null   object
 16  Power              7509 non-null   object


In [546]:
cazoo_cars.isnull().sum()

Name                   0
Year                  11
Mileage               11
Engine_Size          139
Transmission          11
Seats                 11
Body_Type             11
Colour                11
Drive_Type            12
Registration          11
Previous_Owners       11
Keys                  11
Emissions             11
ULEZ_Exempt           15
Top_speed             11
Accelaration_0_62     11
Power                 11
Tax                   11
Insurance             11
Mpg                  145
Service_record         0
Price                  0
dtype: int64

In [548]:
missing_col = cazoo_cars.drop(columns = ['Service_record','Price','Name']) # dataframe with columns containing no Nan
all_NaN = missing_col.isnull().all(1) #row indexes where all columns in missing_col contain NaN
name_Nans = cazoo_cars.loc[all_NaN, "Name"] # name values for rows with all NaN
missing_col[all_NaN].join(name_Nans).head()

,Year,Mileage,Engine_Size,Transmission,Seats,Body_Type,Colour,Drive_Type,Registration,Previous_Owners,Keys,Emissions,ULEZ_Exempt,Top_speed,Accelaration_0_62,Power,Tax,Insurance,Mpg,Name
2300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Looks like this page is missing
2656,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Looks like this page is missing
3930,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Looks like this page is missing
4332,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Looks like this page is missing
4918,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Looks like this page is missing


In [547]:
all_NaN

0       False
1       False
2       False
3       False
4       False
        ...  
7515    False
7516    False
7517    False
7518    False
7519    False
Length: 7520, dtype: bool

In [549]:
cazoo_cars.drop(cazoo_cars[all_NaN].index, inplace = True)
cazoo_cars.reset_index(drop = True, inplace = True)

Where all car detail values are missing the car has been sold, before the page was scraped, wiping out the data. However cases where details such as mpg and engineSize are missing, this because the car is electric as seen below.

Cases where ULEZ expmtion is not possible for a car contains NaN values

In [550]:
missing_col = cazoo_cars.loc[:,['Name','ULEZ_Exempt']] # dataframe with columns containing no Nan
any_NaN = missing_col.isnull().any(1) #row indexes where all columns in imp_col contain NaN
name_Nans = cazoo_cars.loc[all_NaN, "Name"] # name values for rows with all NaN
missing_col[any_NaN]

,Name,ULEZ_Exempt
3209,Hyundai i10,NaN
6880,Toyota C-HR,NaN
6922,Hyundai TUCSON,NaN
7191,Audi Q5,NaN


In [551]:
missing_col = cazoo_cars.loc[:,['Name','Drive_Type']] # dataframe with columns containing no Nan
any_NaN = missing_col.isnull().any(1) #row indexes where all columns in imp_col contain NaN
name_Nans = cazoo_cars.loc[all_NaN, "Name"] # name values for rows with all NaN
missing_col[any_NaN]

,Name,Drive_Type
6849,Lexus UX 250h,NaN


only one case where drive type has a NaN value, in this case the particular car is front wheel drive this will be imputed.

In [552]:
missing_engine_col = cazoo_cars.loc[:,['Name','Engine_Size']] # dataframe with columns
any_NaN = missing_engine_col.isnull().any(1) #row indexes where all columns in imp_col contain NaN
name_Nans = cazoo_cars.loc[any_NaN, "Name"] # name values for rows with NaN
engine_missing = missing_engine_col[any_NaN]

In [553]:
engine_missing["Name"].unique()

array(['Nissan Leaf', 'Smart fortwo', 'Renault Zoe', 'Hyundai IONIQ',
       'Seat Mii', 'BMW i3', 'Kia Soul', 'MG MG ZS', 'Volkswagen Golf',
       'Vauxhall Corsa-e', 'Mazda MX-30', 'Volkswagen e-Golf',
       'Peugeot e-208', 'Peugeot 208', 'Fiat 500e', 'MG MG5',
       'Fiat 500 Electric', 'Mini Electric', 'Hyundai KONA',
       'Vauxhall Mokka', 'Jaguar I-PACE', 'Tesla Model 3',
       'Audi Q4 e-tron', 'Tesla Model S', 'Audi e-tron'], dtype=object)

Looking at unique values, all cars with engine size missing are electric cars

In [554]:
missing_mpg_col = cazoo_cars.loc[:,['Name','Mpg']] # dataframe with columns
any_NaN = missing_mpg_col.isnull().any(1) #row indexes where all columns in imp_col contain NaN
name_Nans = cazoo_cars.loc[any_NaN, "Name"] # name values for rows with NaN
mpg_missing = missing_mpg_col[any_NaN]

In [555]:
mpg_missing["Name"].unique()

array(['Nissan Leaf', 'Smart fortwo', 'Renault Zoe', 'Hyundai IONIQ',
       'Seat Mii', 'BMW i3', 'Kia Soul', 'MG MG ZS', 'Volkswagen Golf',
       'Vauxhall Corsa-e', 'Mazda MX-30', 'Mini Countryman',
       'Volkswagen e-Golf', 'Peugeot e-208', 'Peugeot 208', 'Fiat 500e',
       'MG MG5', 'BMW 5 Series', 'Fiat 500 Electric', 'Mini Electric',
       'Hyundai KONA', 'Vauxhall Mokka', 'Jaguar I-PACE', 'Tesla Model 3',
       'Audi Q4 e-tron', 'Tesla Model S', 'Audi e-tron'], dtype=object)

All Mpg missing values also corrospond to electric cars

In [571]:
cazoo_cars.isnull().sum()

Brand                  0
Make                   0
Year                   0
Mileage                0
Engine_Size          128
Transmission           0
Seats                  0
Body_Type              0
Colour                 0
Drive_Type             1
Registration           0
Previous_Owners        0
Keys                   0
Emissions              0
ULEZ_Exempt            0
Top_speed              8
Accelaration_0_62    265
Power                  0
Tax                    0
Insurance              0
Mpg                  134
Service_record         0
Price                  0
dtype: int64

In [557]:
cazoo_cars["ULEZ_Exempt"].unique()

array(['YesLearn more', 'NoLearn more', nan], dtype=object)

In [558]:
cazoo_cars["ULEZ_Exempt"] = cazoo_cars["ULEZ_Exempt"].fillna(0) #In ULEZ column change NaN to 0 
for i in range(0,len(cazoo_cars)):
    if cazoo_cars.loc[i,"ULEZ_Exempt"] == 'YesLearn more':
        yes = cazoo_cars.loc[i,"ULEZ_Exempt"].replace(cazoo_cars.loc[i,"ULEZ_Exempt"],'1')
        cazoo_cars.loc[i,"ULEZ_Exempt"] = int(yes)
    elif cazoo_cars.loc[i,"ULEZ_Exempt"] == 'NoLearn more':
        no = cazoo_cars.loc[i,"ULEZ_Exempt"].replace(cazoo_cars.loc[i,"ULEZ_Exempt"],'0')
        cazoo_cars.loc[i,"ULEZ_Exempt"] = int(no)
        
cazoo_cars["ULEZ_Exempt"] = cazoo_cars["ULEZ_Exempt"].astype("int64")

In [559]:
cazoo_cars.head()

,Name,Year,Mileage,Engine_Size,Transmission,Seats,Body_Type,Colour,Drive_Type,Registration,...,Emissions,ULEZ_Exempt,Top_speed,Accelaration_0_62,Power,Tax,Insurance,Mpg,Service_record,Price
0,Kia Picanto,Dec 2015,"54,990 miles",1L,Manual,5,3 door Hatchback,White,Front wheel drive,NV65 AZJ,...,105g/km (band B),1,98 mph,14.4 seconds,66 bhp,£20 / year,Group 2 (approx. £215 / year),62.80mpg (approx. £18 / week),6,"£5,700"
1,Dacia Sandero,Mar 2016,"24,138 miles",0.9L,Manual,5,5 door Hatchback,Chestnut,Front wheel drive,SJ16 VWX,...,109g/km (band B),1,109 mph,11.1 seconds,89 bhp,Not available,Group 8 (approx. £208 / year),57.70mpg (approx. £19 / week),0,"£6,250"
2,Ford Ka,Sept 2015,"30,571 miles",1.2L,Manual,4,3 door Hatchback,Black,Front wheel drive,EO65 UXD,...,115g/km (band C),1,99 mph,13.4 seconds,68 bhp,Not available,Group 3 (approx. £244 / year),57.70mpg (approx. £19 / week),3,"£6,300"
3,Fiat Punto,Sept 2016,"38,799 miles",1.2L,Manual,5,5 door Hatchback,Ambient White,Front wheel drive,FY66 BPF,...,124g/km (band D),1,97 mph,14.4 seconds,69 bhp,Not available,Group 8 (approx. £283 / year),53.30mpg (approx. £21 / week),5,"£6,400"
4,Volkswagen up!,Jul 2016,"58,887 miles",1L,Manual,4,3 door Hatchback,Yellow,Front wheel drive,FC16 ZFX,...,105g/km (band B),1,99 mph,14.4 seconds,59 bhp,Not available,Group 2 (approx. £262 / year),62.80mpg (approx. £18 / week),7,"£6,450"


In [560]:
names = cazoo_cars["Name"]

brand = []
make = []
for i in names:
    s = i.split(" ")
    if len(s) == 2:
        b = s[0]
        m = s[1]
    elif len(s) ==3:
        b = s[0]
        m = ' '. join(s[1:])
    elif len(s) == 4:
        if "Land Rover" in ' '.join(s):
            b = ' '.join(s[:2])
            m = ' '.join(s[1:])
        else:
            b = s[0]
            m = ' '.join(s[1:])
    elif len(s) == 5:
        if "Land Rover" in ' '.join(s):
            b = ' '.join(s[:2])
            m = ' '.join(s[1:])
        else:
            b = s[0]
            m = ' '.join(s[1:])
    brand.append(b)
    make.append(m)

b_df = pd.DataFrame({'Brand':brand})
m_df = pd.DataFrame({'Make':make})

b_m = pd.concat([b_df,m_df], axis = 1)
cazoo_cars = pd.concat([b_m, cazoo_cars], axis = 1)
cazoo_cars = cazoo_cars.drop(columns = "Name")

In [561]:
def cleaner(df,pattern, col_name, t):
    
    for i in range(0,len(df)):
        univ_key = {ord('£'): None, ord(','):None} # the'universal key' symbols common to all columns that should be removed
        if pd.notna(df.loc[i,col_name]): # A 'NaN' value will throw an error on re.search()so exclude
            check= re.search(pattern, df.loc[i, col_name]) #search for pattern and extract useful information
            if check == None:
                df.loc[i,col_name] = np.NaN
            else:
                key = check.group()
                new = key.translate(univ_key) # remove symbols with univ_key
                df.loc[i,col_name] = t(new)#replace old with new
        
        else: 
            continue # skips to next when a NaN value is reached

In [562]:
cleaner(cazoo_cars,'\d*([\,]|\d)*\d','Price',int)
cleaner(cazoo_cars,'\d{0,2}(\,)?\d+', 'Mileage',int)
cleaner(cazoo_cars,'\d(\.\d)?', 'Engine_Size',float)
cleaner(cazoo_cars,'\d{3}\d', 'Year',int)
cleaner(cazoo_cars,'\d+\d', 'Top_speed',int)
cleaner(cazoo_cars,'\d+(\.\d+)', 'Mpg',float)
cleaner(cazoo_cars,'\d{0,2}(\.)?(\d)', 'Accelaration_0_62',float)
cleaner(cazoo_cars,'\d+(\.)?\d', 'Power',float)
cleaner(cazoo_cars,'Group\s[0-9]', 'Insurance',str)
cleaner(cazoo_cars,'[0-9]', 'Keys',int)
cleaner(cazoo_cars,'[0-9]', 'Keys',int)

In [581]:
cleaner(cazoo_cars,'\d', 'Insurance',int)

In [569]:
check =cazoo_cars['Accelaration_0_62'][112]
check

nan

In [568]:
cazoo_cars['Accelaration_0_62'][gimme]

Series([], Name: Accelaration_0_62, dtype: object)

In [565]:
gimme = []
for i in range(0,len(cazoo_cars)):
    if pd.notna(cazoo_cars.loc[i, 'Accelaration_0_62']):
        f = re.search('\d{0,2}(\.)?(\d)',cazoo_cars.loc[i, 'Accelaration_0_62'], )
        if f == None:
            gimme.append(i)
        else:
            print(f.group())
       
    else:
        continue

TypeError: expected string or bytes-like object

In [566]:
cazoo_cars.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7509 entries, 0 to 7508
Data columns (total 23 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Brand              7509 non-null   object
 1   Make               7509 non-null   object
 2   Year               7509 non-null   object
 3   Mileage            7509 non-null   object
 4   Engine_Size        7381 non-null   object
 5   Transmission       7509 non-null   object
 6   Seats              7509 non-null   object
 7   Body_Type          7509 non-null   object
 8   Colour             7509 non-null   object
 9   Drive_Type         7508 non-null   object
 10  Registration       7509 non-null   object
 11  Previous_Owners    7509 non-null   object
 12  Keys               7509 non-null   object
 13  Emissions          7509 non-null   object
 14  ULEZ_Exempt        7509 non-null   int64 
 15  Top_speed          7501 non-null   object
 16  Accelaration_0_62  7244 non-null   object


In [577]:
cazoo_cars['Insurance'].unique()

array(['Group 2', 'Group 8', 'Group 3', 'Group 1', 'Group 6', 'Group 9',
       'Group 4', 'Group 5', 'Group 7'], dtype=object)

In [582]:
cazoo_cars.to_csv("Cazoo_new.csv", index=False)

In [579]:
gimme = []
for i in range(0,len(cazoo_cars)):
    if pd.notna(cazoo_cars.loc[i, 'Insurance']):
        f = re.search('\d',cazoo_cars.loc[i, 'Insurance'], )
        if f == None:
            gimme.append(i)
        else:
            print(f.group())
       
    else:
        continue

2
8
3
8
2
1
6
9
2
4
1
5
7
2
7
2
1
7
1
1
2
3
3
1
4
3
2
8
6
2
1
7
7
7
7
2
8
8
8
8
1
8
2
1
1
1
5
8
7
5
5
7
8
1
8
3
7
1
2
4
8
8
7
8
2
2
2
5
7
8
7
8
3
1
1
9
5
1
2
6
4
4
1
7
1
1
3
8
1
2
6
1
2
5
6
1
1
4
7
7
2
4
6
2
8
7
4
4
1
1
7
3
1
1
8
7
7
7
6
8
1
1
7
1
3
7
7
7
7
7
5
1
7
1
1
2
1
2
1
1
2
1
1
2
1
3
8
9
7
7
1
8
1
1
9
6
2
2
7
7
8
1
1
1
3
1
8
7
2
4
3
1
1
7
1
4
2
9
8
1
1
1
6
1
5
7
2
3
2
1
3
4
2
1
1
1
9
1
1
8
3
3
7
1
1
8
5
3
8
1
1
7
1
8
7
3
1
7
1
1
6
8
8
7
1
6
3
1
6
2
7
9
1
1
7
7
4
6
2
7
9
1
8
3
1
8
8
8
9
8
3
8
2
9
9
1
9
3
8
7
1
3
1
8
4
7
7
6
1
7
1
1
3
9
4
9
2
9
1
1
1
1
3
1
3
8
1
3
1
1
1
2
1
1
1
1
7
1
1
2
2
7
8
1
7
1
7
1
8
4
4
8
7
8
1
1
7
9
8
1
1
1
4
3
2
7
1
6
6
1
1
1
2
1
1
5
1
5
8
6
1
8
3
1
9
9
1
1
9
3
1
1
1
6
8
6
1
6
8
5
8
1
1
8
6
2
1
5
1
9
1
1
1
1
6
1
1
6
1
1
1
5
2
8
6
8
7
6
1
3
3
1
1
2
8
1
9
3
3
5
1
1
1
5
8
1
6
9
2
8
2
2
8
8
1
8
1
8
8
8
8
9
1
1
1
1
2
7
5
1
1
9
1
1
3
6
7
9
6
1
1
1
1
1
8
1
1
1
1
1
7
1
9
1
2
1
8
1
6
1
5
9
7
1
4
2
7
1
1
8
1
2
1
1
1
1
2
1
1
1
8
1
3
4
9
4
1
1
1
8
1
6
1
1
2
1
1
8
8
5


gimme

In [580]:
gimme

[]